## This notebook is meant to introduce the basic functionality of python packages that will be used throughout the course

### Qiskit
#### A quantum computing package that will be used in the quantum algorithm sections. It has modules for Grover's seach and Shor's factoring algorithms as well as tools that will be helpful for quantum simulation and measurement schemes. This will be the package used to run algorithms on simulators as well as real quantum computers through the cloud

https://qiskit.org/documentation/

https://github.com/Qiskit

### Quimb
#### A tensor network package that will be used in the entanglement sections. It has general quantum information tools in the tensor network language such as partial traces and measures of entanglement. There are also numerical tools for matrix product states, other tensor network ansatz such as MERA, and quantum circuit optimization.

https://quimb.readthedocs.io/en/latest/index.html

https://github.com/jcmgray/quimb

### Other packages that will be used
#### numpy: standard linear algebra package for python, necessary for most other things - basic objects are multidimensional arrays
#### matplotlib: standard data visualization for python

## Qiskit

### Basic objects
#### QuantumCircuits
An object that holds a number of qubits and instructions for what gates to apply to them. It also holds classical bits to record the outcomes of qubit measurements.

#### Backends
These are the backends to run your circuits on. Usually you'll run on a simulator but also lets you run on a real device if you generate credentials by making an account here: https://quantum-computing.ibm.com/

In [ ]:
# Imports
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram

### Here we'll see the see the circuit to create and measure a Bell pair

In [ ]:
# Use Aer's qasm_simulator
simulator = Aer.get_backend('qasm_simulator')

# Create a Quantum Circuit acting on the q register with 2 qubits and 2 classical bits
circuit = QuantumCircuit(2, 2)

# Add a H gate on qubit 0
circuit.h(0)

# Add a CX (CNOT) gate on control qubit 0 and target qubit 1
circuit.cx(0, 1)

# Other gates you can apply - Note the 1 & 2 qubit gates are universal 
# meaning that any unitary can be expressed as a product of them
#circuit.x(0) # apply Pauli X to qubit 0
#circuit.rx(0.1,1) # apply a X rotation by angle 0.1 to qubit 1
#circuit.t(0) # apply a T gate to qubit 0


# Map the quantum measurement to the classical bits
circuit.measure([0,1], [0,1])

# Execute the circuit on the qasm simulator
job = execute(circuit, simulator, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(circuit)
print("\nTotal count for 00 and 11 are:",counts)

# Draw the circuit
circuit.draw()

## Quimb

### Basic objects

### Tensor
A Multidimensional array with labeled indices that can be contracted with indices from other tensors - the Legos of quantum objects

### TensorNetwork
A collection of Tensors to be contracted or further manipulated

In [ ]:
#imports
import quimb as qu
import quimb.tensor as qtn

### Here we'll create a tensor network corresponding to an inner product Bell pairs with Paulis applied and a random 2-qubit state

In [ ]:
data = qu.bell_state('phi+').reshape(2, 2) # data array used to create the tensor made form a built in bell state
inds = ('k0', 'k1') # named indices are used to keep track of which legs are connected to what
tags = {'KET'} # tags are handles

ket = qtn.Tensor(data, inds, tags) 
ket.graph(color=['KET'], figsize=(4, 4))

In [ ]:
# Create Paulis and specify where they are being applied through the use of the named indices 'k0' and 'k1' on 'KET'
X = qtn.Tensor(qu.pauli('X'), inds=('k0', 'b0'), tags={'PAULI', 'X', '0'})
Y = qtn.Tensor(qu.pauli('Y'), inds=('k1', 'b1'), tags={'PAULI', 'Y', '1'})

# Create a Bra 
bra = qtn.Tensor(qu.rand_ket(4).reshape(2, 2), inds=('b0', 'b1'), tags={'BRA'})

# Create the TensorNetwork by connecting the tensors together with the & sign. Legs are matched up by index name by
TN = ket.H & X & Y & bra
TN.graph(color=['KET', 'PAULI', 'BRA'], figsize=(4, 4))
print(f'={TN^all}') # ^ contracts the desired indices in this case all

### Other packages that may interest you
#### QuTip: General purpose quantum mechanics package, has various tools for perturbation theory, optimal control, open systems
#### OpenFermion: Alternative software to map from fermionic systems to qubits